# RNA Velocity analysis of p036 colon

In [1]:
%%capture
#@title Install packages
#!pip install matplotlib
#!pip install scikit-learn
#!pip install numpy
#!pip install scipy
#!pip install anndata
#!pip install mizani
#!pip install plotnine
#!pip install -U scvelo
import numpy as np
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import matplotlib.colors as mplcol
import matplotlib.font_manager
import matplotlib as mpl
import pandas as pd
import io
import anndata
import os
import json

from scipy.stats import binom
from scipy.stats import poisson
from scipy.sparse import csr_matrix
from scipy.io import mmread
from sklearn import linear_model

from IPython.display import HTML
from mizani.breaks import date_breaks
from mizani.formatters import date_format
# Only pandas >= v0.25.0 supports column names with spaces in querys
import plotnine as p
import requests
import warnings
import colorsys
warnings.filterwarnings("ignore")  # plotnine has a lot of MatplotlibDeprecationWarning's
import seaborn as sns
sns.set_context("notebook", font_scale=1.5, rc={"lines.linewidth": 2.5})

fsize=20

plt.rcParams.update({'font.size': fsize})
%config InlineBackend.figure_format = 'retina'

## following: https://github.com/basilkhuder/Seurat-to-RNA-Velocity

In [ ]:
#%load_ext rpy2.ipython

## Loading data

In [3]:
# define path as csv file to all input files
rawdir = "/misc/data/rawData/RNA/singleCell/5prime/"
samples = "/misc/data/user/pun06483/projects/pr036_dd_10x/analysispy/RNAVelocity/metainfo/samples.txt"
fqsj = "/misc/data/user/pun06483/projects/pr036_dd_10x/analysispy/RNAVelocity/metainfo/output.json"
outdir = "/misc/data/user/pun06483/projects/pr036_dd_10x/analysispy/RNAVelocity"
outdir2 = "/misc/data/user/pun06483/projects/pr036_dd_10x/analysispy/RNAVelocity2"
gtf = "/misc/software/ngs/cellranger/refdata/refdata-gex-mm10-2020-A/genes/genes.gtf"
fasta = "/misc/software/ngs/cellranger/refdata/refdata-gex-mm10-2020-A/fasta/genome.fa"

In [4]:
#paths
indx = os.path.join(outdir,"transcriptome.idx")
t2genes = os.path.join(outdir,"t2g.txt")
cdna = os.path.join(outdir,"cdna.fa")
intron = os.path.join(outdir,"intron.fa")
# transcript to capture
cdnat2c = os.path.join(outdir, "cdna_t2c.txt")
intron_t2c = os.path.join(outdir,"intron_t2c.txt")


Index creation for kallisto

In [ ]:
print(indx)
print(cdna)
!kb ref -h

In [ ]:
# shell calls must start with "!" and variables are passed via "{variable}"
#!kb ref -i {indx} -g {t2genes} -f1 {cdna} -f2 {intron} -c1 {cdnat2c} -c2 {intron_t2c} --workflow lamanno {fasta} {gtf} 
#$ kb count -i index.idx -g t2g.txt -x 10xv2 --h5ad -t 2 read_1.fastq.gz read_2.fastq.gz

In [ ]:
print(indx)
os.path.exists(indx)

In [ ]:
########################
## DO NOT RUN
#################


#%%R -i rawdir -i samples #-w 5 -h 5 --units in -r 200
# import df from global environment: -i df
# make default figure size 5 by 5 inches with 200 dpi resolution
# handle multiple input files
# rbioc_3-14
#library(rjson)

#rawdir <- "/misc/data/rawData/RNA/singleCell/5prime"
#samples <- "/misc/data/user/pun06483/projects/pr036_dd_10x/analysispy/RNAVelocity/metainfo/samples.txt"
#samples <- read.delim(samples,header=F)[,1]

# find all fastq.gz and select the sample respectively
#fls <- list.files(rawdir,pattern="fastq.gz",full.names=T,recursive=T)
#fls <- fls[grep("_I[1,2]_",fls,invert=T)]
#indx<-lapply(samples,function(x) grep(x,fls))
#flsl <- lapply(indx,function(x) fls[x])

# get samples paired for R1 and R2
#flsl <- lapply(flsl,function(x) split(x,dirname(x)))
#names(flsl) <- samples
#library(rjson)               
# read list ot json
#jsonData <- toJSON(flsl)
 
# write json object to file
#write(jsonData, "/misc/data/user/pun06483/projects/pr036_dd_10x/analysispy/RNAVelocity/metainfo/output.json")



In [5]:
with open(fqsj,"rb") as fqsh:
    fqs = json.load(fqsh)
samples=list()
for sample in fqs.keys():
    samples.append(sample)

print(samples)

['RNA_2_C_ra', 'RNA_3_C_ra', 'RNA_4_C_ra']


In [6]:
!kb --list

List of supported single-cell technologies

Positions syntax: `input file index, start position, end position`
When start & end positions are None, refers to the entire file
Custom technologies may be defined by providing a kallisto-supported technology string
(see https://pachterlab.github.io/kallisto/manual)

name         description                       whitelist    barcode                    umi        cDNA                       
---------    ------------------------------    ---------    -----------------------    -------    -----------------------    
10XV1        10x version 1                     yes          0,0,14                     1,0,10     2,None,None                
10XV2        10x version 2                     yes          0,0,16                     0,16,26    1,None,None                
10XV3        10x version 3                     yes          0,0,16                     0,16,28    1,None,None                
BDWTA        BD Rhapsody                       yes       

In [9]:
with open(fqsj,"rb") as fqsh:
    fqs = json.load(fqsh)
    
for sample in fqs.keys():
    print(sample)

print(outdir2)

RNA_2_C_ra
RNA_3_C_ra
RNA_4_C_ra
/misc/data/user/pun06483/projects/pr036_dd_10x/analysispy/RNAVelocity2


In [ ]:

##############################
## DO NOT RERUN##########
##########################


# merge reads into one file for each sample
#!kb count -h
#!kb --list

# kb velocity command,
# lamanno: calculation of RNA velocity
# loom: special file type H5a is also fine, loom needed for interaction with oterh programms
# x: for which technology: select from kb --list
# # zcat your.fastq.gz|head -n 4|tail -n 1|wc -c # minus 1 as line end is counted!!!
## in our case R1 has length 28 so 10xv3

import json


from subprocess import check_call



samples=list()
for sample in fqs.keys():
    samples.append(sample)
    outdirSample = os.path.join(outdir2,sample)
    outstr = ""
    skeys = fqs.get(sample).keys()
    fqsu=fqs.get(sample)
    for sk in skeys:
        #print(fqsu.get(sk))
        outstr = " ".join([outstr]+fqsu.get(sk))
        outstr = outstr.lstrip()
    
    print(outstr)
    print("Processing: ",sample)
    cmd=" ".join(["kb","count",
               "-i",indx,
               "-t", "12",
               "-x", "10xv3",
               "-o", outdirSample,
               "--workflow","lamanno",
               "--loom",
               "-g",t2genes,
               "-c1",cdnat2c,
               "-c2", intron_t2c,
               outstr])
    check_call(cmd,shell=True)
    
    #           "kb","count",
    #           "-i",indx,
    #          "-t", "12",
    #           "-x", "10xv3",
    #           "-o", outdirSample,
    #           "--workflow","lamanno",
    #           "--loom",
    #           "-g",t2genes,
    #           "-c1",cdnat2c,
    #           "-c2", intron_t2c,
    #           outstr],shell=False,check=True)




/misc/data/rawData/RNA/singleCell/5prime/BSF_0820_1/SCRNA_SEQ_082020_1_L4439/RNA_2_C_ra/RNA_2_C_ra_S3_L001_R1_001.fastq.gz /misc/data/rawData/RNA/singleCell/5prime/BSF_0820_1/SCRNA_SEQ_082020_1_L4439/RNA_2_C_ra/RNA_2_C_ra_S3_L001_R2_001.fastq.gz /misc/data/rawData/RNA/singleCell/5prime/BSF_0820_2/SCRNA_SEQ_082020_1_L4439/RNA_2_C_ra/RNA_2_C_ra_S6_L002_R1_001.fastq.gz /misc/data/rawData/RNA/singleCell/5prime/BSF_0820_2/SCRNA_SEQ_082020_1_L4439/RNA_2_C_ra/RNA_2_C_ra_S6_L002_R2_001.fastq.gz /misc/data/rawData/RNA/singleCell/5prime/BSF_0850_1/P036_SCRNA_SEQ_2_L4535/RNA_2_C_ra/RNA_2_C_ra_S2_L001_R1_001.fastq.gz /misc/data/rawData/RNA/singleCell/5prime/BSF_0850_1/P036_SCRNA_SEQ_2_L4535/RNA_2_C_ra/RNA_2_C_ra_S2_L001_R2_001.fastq.gz /misc/data/rawData/RNA/singleCell/5prime/BSF_0850_2/P036_SCRNA_SEQ_2_L4535/RNA_2_C_ra/RNA_2_C_ra_S8_L002_R1_001.fastq.gz /misc/data/rawData/RNA/singleCell/5prime/BSF_0850_2/P036_SCRNA_SEQ_2_L4535/RNA_2_C_ra/RNA_2_C_ra_S8_L002_R2_001.fastq.gz
Processing:  RNA_2_C_ra


[2022-05-25 08:39:58,212]    INFO [count_lamanno] Using index /misc/data/user/pun06483/projects/pr036_dd_10x/analysispy/RNAVelocity/transcriptome.idx to generate BUS file to /misc/data/user/pun06483/projects/pr036_dd_10x/analysispy/RNAVelocity2/RNA_2_C_ra from
[2022-05-25 08:39:58,212]    INFO [count_lamanno]         /misc/data/rawData/RNA/singleCell/5prime/BSF_0820_1/SCRNA_SEQ_082020_1_L4439/RNA_2_C_ra/RNA_2_C_ra_S3_L001_R1_001.fastq.gz
[2022-05-25 08:39:58,213]    INFO [count_lamanno]         /misc/data/rawData/RNA/singleCell/5prime/BSF_0820_1/SCRNA_SEQ_082020_1_L4439/RNA_2_C_ra/RNA_2_C_ra_S3_L001_R2_001.fastq.gz
[2022-05-25 08:39:58,213]    INFO [count_lamanno]         /misc/data/rawData/RNA/singleCell/5prime/BSF_0820_2/SCRNA_SEQ_082020_1_L4439/RNA_2_C_ra/RNA_2_C_ra_S6_L002_R1_001.fastq.gz
[2022-05-25 08:39:58,213]    INFO [count_lamanno]         /misc/data/rawData/RNA/singleCell/5prime/BSF_0820_2/SCRNA_SEQ_082020_1_L4439/RNA_2_C_ra/RNA_2_C_ra_S6_L002_R2_001.fastq.gz
[2022-05-25 08:

## Extrackting Meta-data

In [ ]:
######################
## DO NOT RERUN ###
#######################


# in r
metadir="/misc/data/user/pun06483/projects/pr036_dd_10x/analysispy/RNAVelocity/metainfo"
#write.csv(Cells(Seurat_object), file = "cellID_obs.csv", row.names = FALSE)
write.csv(Cells(SOcol), file = file.path(metadir,"cellID_colon.csv"), row.names = FALSE)
#write.csv(Embeddings(seurat_object, reduction = "umap"), file = "cell_embeddings.csv")
write.csv(Embeddings(SOcol, reduction = "umap"), file = file.path(metadir,"cell_colon_embeddings.csv"))
#write.csv(seurat_object@meta.data$seurat_clusters, file = "clusters.csv")
outf<-SOcol@meta.data$seurat_clusters

ggplotColours <- function(n = 6, h = c(0, 360) + 15){
  if ((diff(h) %% 360) < 1) h[2] <- h[2] - 360/n
  hcl(h = (seq(h[1], h[2], length = n)), c = 100, l = 65)
}

color_list <- ggplotColours(n=8)
#source: https://stackoverflow.com/questions/8197559/emulate-ggplot2-default-color-palette
# source: https://github.com/satijalab/seurat/issues/2366
outdf=data.frame("cluster"=outf,
                "col"=outf)
levels(outdf$col)=color_list
write.csv(outdf, file = file.path(metadir,"colon_clusters.csv"))


In [ ]:
import anndata
import scvelo as scv
import pandas as pd
import numpy as np
import matplotlib as plt
#%load_ext rpy2.ipython

print(samples)
sample_two = anndata.read_loom("/misc/data/user/pun06483/projects/pr036_dd_10x/analysispy/RNAVelocity/RNA_3_C_ra/counts_unfiltered/adata.loom")

metadir="/misc/data/user/pun06483/projects/pr036_dd_10x/analysispy/RNAVelocity/metainfo"
sample_obs = pd.read_csv(os.path.join(metadir,"cellID_colon.csv"))
umap_cord = pd.read_csv(os.path.join(metadir,"cell_colon_embeddings.csv"))
cell_clusters = pd.read_csv(os.path.join(metadir,"colon_clusters.csv"))



In [ ]:
dir(sample_one)
print(type(sample_one))
print(type(sample_one.obs.barcode))
print(sample_one.obs.loc[:,"barcode"])
#"".join(["2C_",sample_one.obs.loc[:,"barcode","-1"])
#df['prod_type'] = 
B=[''.join(['2C_',item,'-1']) for item in sample_one.obs['barcode']]
#C=deepcopy(sample_one)
#C.obs["barcode"]=B
#print(C.obs["barcode"])

In [ ]:
IDs=[x.replace('RNA_','') for x in samples]
IDs=["".join([x[0:1],"C"]) for x in IDs]
sample_pddf = {'names':samples,
               'IDs':IDs}
sample_pddf = pd.DataFrame(sample_pddf)


print(str(sample_pddf.loc[sample_pddf['names']=='RNA_2_C_ra',"IDs"]))
print(sample_pddf)
sample_obs

In [ ]:
print(samples)
anndatl = list()
sample_obs[1:10]
#for sample in samples:
    #sample="RNA_2_C_ra"
    #stpath = "/misc/data/user/pun06483/projects/pr036_dd_10x/analysispy/RNAVelocity"
    #spath = os.path.join(stpath,sample,"counts_unfiltered/adata.loom")
    #print(spath)
    #ind = anndata.read_loom(spath)
    #print(ind)
    #pattern = sample_pddf.loc[sample_pddf['names'] == sample,'IDs'].item()
    #print(pattern)
    #ind.obs['barcode']=[''.join([pattern,'_',item,'-1']) for item in ind.obs['barcode']]
    ##cellID_obs = sample_obs[sample_obs['x'].str.contains(pattern,regex=False)]
    ##cellID_obs['x'] = cellID_obs['x'].replace("".join([pattern,"_"]),"",regex=True)
    ##cellID_obs['x'] = cellID_obs['x'].replace('-1',"",regex=True)                                          
    #print(len(cellID_obs))
    #outf = ind[np.isin(ind.obs.barcode,cellID_obs)]
    #print(outf)
    #anndatl.append(outf)



In [ ]:
annd=anndatl[0].concatenate(anndatl[1:])

In [ ]:
annd.obs.barcode
print(len(annd.obs.barcode))

Now the velocity data is filtered based upon the Seurat object. We end up with only 1515 cells of 14780 in the seurat data set. Is this due to barcode correction errors between cellrange and kb tools?
Now we ad umap coordinates


In [ ]:
umap_cord[:2]

In [ ]:
annd_index = pd.DataFrame(annd.obs.barcode)
annd_index = annd_index.rename(columns={'barcode':'Cell ID'})
annd_index[:2]

In [ ]:
umap_cord = umap_cord.rename(columns = {'Unnamed: 0':'Cell ID'})
umap_cord[:2]

In [ ]:
umap_ordered = annd_index.merge(umap_cord, on = "Cell ID")
umap_ordered[:2]
print(umap_ordered.max())
print(umap_ordered.min())


In [ ]:
now the order of the umap is correct, add it now without the cell ID to the annd

In [ ]:
umap_ordered = umap_ordered.iloc[:,1:]
print(umap_ordered.values)
annd.obsm['X_umap'] = umap_ordered.values

In [ ]:
cell_clustersn = cell_clusters.join(umap_cord)
cell_clustersn = cell_clustersn[["Cell ID","cluster","col"]]
cell_cluster_ordered = annd_index.merge(cell_clustersn, on = "Cell ID")
cell_cluster_ordered[:2]
colors_ord = cell_cluster_ordered.iloc[:,2]
cluster_ord = cell_cluster_ordered.iloc[:,1]


In [ ]:
cell_cluster_ordered.astype("object").dtypes
cell_cluster_ordered.astype({'cluster': 'object'}).dtypes
cell_cluster_ordered.dtypes

In [ ]:
annd.uns['Cluster_colors'] = colors_ord.values
annd.uns['clusters'] = pd.Categorical(cluster_ord.values.astype(str))
annd.uns['clusters']

In [ ]:
annd.obs['clusters']=pd.Categorical(cluster_ord.values.astype(str))

Run RNA Verlocity


In [ ]:
annd

In [ ]:
scv.pl.proportions(annd)

In [ ]:
scv.pp.filter_and_normalize(annd)
scv.pp.moments(annd)
scv.tl.velocity(annd, mode = "stochastic",vkey="stochastic")

scv.tl.velocity_graph(annd,vkey="sto")
scv.pl.velocity_embedding(annd, basis = 'umap',
                          save = os.path.join(stpath,"ScRNAVelo.pdf"))

In [ ]:
annd.write(os.path.join(stpath,'anndata_colon_p36_scvelo.h5ad'), compression="gzip")

In [ ]:
annd.obs

In [ ]:
annd.uns['Cluster_colors']

In [ ]:
print(np.max(annd.obsm["X_umap"],axis=0))
print(np.min(annd.obsm["X_umap"],axis=0))


In [ ]:
scv.pl.velocity_embedding(annd, basis = 'umap',
                          scale = 4,arrow_length=2,xlim=(-6,6),ylim=(-6,6),
                          show=True,
                          save = os.path.join(stpath,"ScRNAVelo.pdf"))


In [ ]:
scv.pl.velocity_embedding

In [ ]:
#kwargs={"xlim":(-6,6),"ylim":(-6,6)}
#print(kwargs)
scv.pl.velocity_embedding(annd, basis = 'umap',legend_loc="right",xlabel="UMAP1",ylabel="UMAP2",
                          figsize=(7,7),color='clusters',colorbar=False,palette=annd.uns['Cluster_colors'],
                          scale = 4,arrow_length=3,arrow_size=2,dpi=150,show=True,
                          save = os.path.join(stpath,"ScRNAVelo.pdf"),xlim=[-6,6],ylim=[-6,6]
                          )



                          
                          

In [ ]:
scv.set_figure_params('scvelo') 
kwargs = dict(color_map='gnuplot', title='',
              vmin=-.1, colorbar=False, show=False, dpi=150)
scv.pl.velocity_embedding_stream(annd, basis = 'umap',
                          xlim=(-6,6),ylim=(-6,6),
                          save = os.path.join(stpath,"ScRNAVelo_stream.pdf"),**kwargs)